# Neural Network implementation from scratch

- Empezar explicando como funciona neural network
- Explicar básicamente matrix multiplication
- Cómo funciona Layer.forward. 
- Cómo estructurar Net
- Cómo funciona SGD
- Cómo funciona backward

This notebook will go through a basic implementation of a neural network unsing only NumPy. And this is the only library that we'll need. Let's import it then. We're defining also a decorator that will help us to add methods to the classes as we make them in the notebook. 

In [13]:
import numpy as np

def add_to_class(Class): 
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

It will go step by step, from the input data to the update of the network weights and biases. It won't go deep into math and derivation, check the Resources section at the bottom of the page to read more about those topics. 

1. Neuron
2. Layer
3. Network
   
## Neuron
A neural network is made out of (yes, you guessed it) neurons. A neuron can basically be described as a linear mathematic function. It takes any number of inputs, it multiplies them by a number we call the neuron's weights, it sums everything, and it adds another number called the bias. For example, a neuron with two inputs looks like this:

$$z(x_1, x_2) = w_1*x_1 + w_2*x_2 + b$$

This number $z$ then passes (most of the times) through an activation function. We're gonna use the ReLU function here, that basically returns the same number that it takes only if that number is positive. Otherwise it returns 0:
$$a = ReLU(z) = max(0, z)$$

> meter links a info de relu

## Layer
One or more neurons with the same inputs form a layer. Here is where the well known layer diagram comes in handy. The upper index on the letter indicates the layer number and the lower index indicates which neuron it is. The weights have a second lower index indicating its order _in its neuron_. For example $w_{34}^2$, would be the fourth weight of the third neuron of the second layer. 

<div style="text-align: center">
    <img src="./img/layer.png" width="600" style="border-radius: 20px"/>
    </br>
    <img src="./img/weight.png" width="200" style="border-radius: 20px"/>
</div>

If we would follow this notation as it is when we code, we would have to calculate the ouptut of each neuron by itself. This is ok if we have 3 neurons, but in modern neural networks we can have hundreds or thousand of neurons, with hundreds of thousands of weights. Apart from this, we will use Stochastic Gradient Descent `meter link a info`, so we'll want to pass several training examples through the network at once. To do that, we'll use matrix operations, that's why we normally save a layer's weights and biases (what we call the parameters) as matrices. In the weights matrix, each column of will represent one neuron, and each row the corresponding weights:

$$\begin{bmatrix} w_{11}^1 & w_{21}^1 & w_{31}^1 \\ w_{12}^1 & w_{22}^1 & w_{32}^1 \end{bmatrix}$$

The bias matrix has only one row, with the biases for each neuron:

$$\begin{bmatrix} b_{1}^1 & b_{2}^1 & b_{3}^1 \end{bmatrix}$$

Let's start coding then. As we're connecting all the inputs to all the weights, we call this a _fully connected_ layer. Sometimes it's also called Linear (if it doesn't have activation function) or Dense layer. To initialize the parameters we'll use `np.random.normal(mean, std_dev, shape)`. If you want to know why we use a standard deviation of $\frac{1}{\sqrt{ins}}$ you can check this book. `meter link al libro de derivadas`. The weights matrix will have a shape of `ins` rows and `outs` columns. The class constructor then looks like this:


In [14]:
class FullyConnected:
    def __init__(self, ins, outs, activation='relu'):
        self.weights = np.random.normal(0, 1/(ins**.5), (ins, outs))
        self.bias = np.random.normal(0, 1/(ins**.5), (1, outs))
        self.activation = activation

## Network
The outputs of a layer become the inputs of the next one. Layers then get connected to each other and that's what makes a network. Let's say that our network has only one more layer, with one more neuron. This will be the output _of the network_, we won't pass it through an activation function in this case. 

<div style="text-align: center">
    <img src="./img/net.png" width="600" style="border-radius: 20px"/>
</div>

To code this, we can simply use an array that we'll call `layers`. The constructor of the class `Net` will accept an array of integers named `neurons` indicating the number of neurons in each layer. `neurons[0]` correspond to the number of inputs. For example, if we have `neurons = (2, 3, 5 ,1)`, we have a neural network with 2 inputs, 3 neurons in the first layer, 5 neurons in the second one, and one output. As we said before, each layer has the same number of inputs as the previous layer has outputs. The code then it's quite self-explanatory:

In [15]:
class Net:
    def __init__(self, neurons, last_relu=False):
        self.layers = []
        for n in range(len(neurons)-1):
            ins = neurons[n]
            outs = neurons[n+1]
            activation = self.decide_activation(len(neurons), last_relu, n)
            layer = FullyConnected(ins, outs, activation=activation)
            self.layers.append(layer)

At line 7 we call the `decide_activation()` method. It's just a helper that sets ReLU as activation to all layers but the last one:

_(we're using the add_to_class decorator we defined above)_

In [16]:
@add_to_class(Net)
def decide_activation(self, neurons, last_relu, n):
        if n == neurons-2 and not last_relu:
            activation = 'none'
        else: 
            activation = 'relu'
        return activation

## Forward pass
We have all the elements we need to make the forward pass. This functions will just take the outputs of one layer and feed them as inputs to the next one, until we have the final input. The code is quite simple: in a for loop we call each layer `forward()` method. We also make a `__call__()`  function so we can directly call the forward pass when we call an instance of the class. 

In [17]:
@add_to_class(Net)
def __call__(self, input):
        return self.forward(input)
    
@add_to_class(Net)
def forward(self, input):
    for layer in self.layers:
        input = layer.forward(input)
    return input

### Multiply the weights by the inputs, and sum everything
As we mentioned above, we want to use the power of matrix multiplications to pass several trainging examples at once. With matrix multiplication the first matrix has to have the same number of rows as the second matrix has columns. Then, we take the first row of the first matrix, multiply its elements by the corresponding elements of the second matrix first column, sum everything, and place the result in the output matrix as a first element. Then we do the same with the second column. 

Let's say we have 5 training examples, with two variables each, and a first layer with 3 neurons. The operation will look like this for the first row: 

$$\begin{bmatrix} x_{1} & y_{1} \\ x_{2} & y_{2} \\ x_{3} & y_{3}\\ x_{4} & y_{4} \\ x_{5} & y_{5}\end{bmatrix} \bullet \begin{bmatrix}w_{11}^1 & w_{21}^1 & w_{31}^1 \\ w_{12}^1 & w_{22}^1 & w_{32}^1\end{bmatrix} = [x_{1}*w_{11}^1 + y_{1}*w_{12}^1 \quad , \quad x_{1}*w_{21}^1 + y_{1}*w_{22}^1\quad , \quad x_{1}*w_{31}^1 + y_{1}*w_{32}^1]$$

Then, we do the same with the remaining rows. As a result, we have a matrix with 5 rows (the same as the original training examples), and 3 columns/variables. Let's see it with actual numbers:

<div style="text-align: center">
    <img src="./img/dotproduct.gif" width="300" style="border-radius: 20px"/>
</div>

Now, we have to add the biases of the layer. As we saw before, for a layer with three neurons the biases are stored as a one row matrix with three columns. When we perform a sum operation between matrix with different shapes, numpy applies broadcasting to the smaller one, creating a compatible matrix internally. If our biases were, for example $[10 \quad 20 \quad 30]$, the operation would look like this: 
$$\begin{bmatrix} 9 & 12 & 15 \\ 19 & 26 & 33 \\ 29 & 40 & 51 \\ 39 & 54 & 69 \\ 49 & 68 & 87 \end{bmatrix}+\begin{bmatrix} 10 & 20 & 30 \\ 10 & 20 & 30 \\10 & 20 & 30 \\10 & 20 & 30 \\10 & 20 & 30 \end{bmatrix}= \begin{bmatrix} 19 & 32 & 45 \\ 29 & 46 & 63 \\ 39 & 60 & 81 \\ 49 & 74 & 99 \\ 59 & 88 & 117 \end{bmatrix}$$
This gives us an ouput matrix with as many rows as we had at the beginning. So we keep having the same number of training examples, but now we have three variables for each example, corresponding to the single output of each neuron.

That gets very sumarized in python: 
```Python
z = np.matmul(input, weights) + bias
```
### Activation function
And now, the only thing left is pass z through the activation function. The ReLU function is performed elementwise (to all the matrix elements) with the following method: 
```Python
a = np.maximum(0, z)
```
All this gets coded in the layer's `forward()` method. We also save the input, the result of the $z$ function, and the final output to calculate the gradients later. 

In [18]:
@add_to_class(FullyConnected)
def forward(self, input):
    self.input = input
    self.z = np.matmul(input, self.weights) + self.bias
    if self.activation == 'relu':
        self.output =  ReLU(self.z)
    else: 
        self.output = self.z
    return self.output

def ReLU(x):
    return np.maximum(0, x)

## Forward pass in action
### Generating fake data
Let's see how this works. We're going to generate fake data with two input variables for this example.

In [19]:
def f(x, y):
    return (x+4)**3 + 4 * y

def generate_data(size):
    X = np.zeros((size, 2))
    Y = np.zeros((size, 1))
    for n in range(size):
        X[n] = np.random.rand(2)
        Y[n] = f(X[n][0], X[n][1])
    return X, Y + np.random.randn(Y.size, 1) * 0.01

X, Y = generate_data(10000)

### Taking a sample
We're gonna use Stochastic Gradient Descent to optimize our network. With this algorithm, in each iteration (called _epoch_), we take a sample of the data (called _minibatch_) with a fixed size and feed the model with it. Then, with the output we calculate the gradients and update the parameters. 

First, we take a sample from the dataset:

In [20]:
BATCH_SIZE = 32
indices = np.random.randint(len(X), size=(BATCH_SIZE))
sample_X = X[indices]
sample_Y = Y[indices]

### Calculate the output and the loss
Now, we create an instance of the net and calculate the output. To really see what's going on behind the curtains, we're making a network with two layers (with 3 neurons the first one and 4 the second one), and one output layer with one neuron. We can see the shape of the output having as many rows as the input, but only one column. In fact, it has the same shape as the true labels, what we called `sample_Y`.


In [21]:
net = Net((2, 3, 4, 1))
out = net(sample_X)
print(f'Input shape: {sample_X.shape}, True labels shape: {sample_Y.shape}, Output shape: {out.shape}')

Input shape: (32, 2), True labels shape: (32, 1), Output shape: (32, 1)


Because `sample_Y` and `out` have the same, shape, we can perform elementwise operations with them. This is very useful to calculate the loss for each training example, and the cost function too. Here we're gonna use the Mean Squared Error loss function.
The error for each example is

$$l_j = \frac{1}{2}(y_j - \^{y}_j)^2$$

where $y$ is the true value and $\^{y}$ is the predicted value. We're dividing by two because it will be much easier to derivate later. 

The _loss_ represents the error of every example. If we want to calculate the error _for the whole network_, we need to calculate a mean of the losses. That's what we call __cost function__:

$$MSE = \frac{1}{2n}\sum_{i=1}^n(y_i - \^{y}_i)^2$$

Let's see the value of the cost function with this first forward pass (expect it to be quite high!):

In [22]:
def MSE(exp, pred):
    return ((exp - pred) ** 2).sum() / (2*len(exp))

MSE(sample_Y, out)

4664.085764379107

## Backpropagation
We're not going to explain why and how backpropagation works. You can check the resources section to look for more detailed information about calculating the partial derivatives for each weight and bias. We're just going to see how to implement it step by step in our neural network. 

What we do in backpropagation is calculate the loss function partial derivatives _backwards_, from the output to every weight and bias of the network. Starting from the last layer, in each one we do the following:
1. Pass the gradients of the next layer.
2. If the layer has activation function, multiply the gradients by the derivative of ReLU.
3. Calculate the gradients of the MSE for every weight.
4. Calculate the gradients of the MSE for every bias.
5. Update the parameters with those gradients.
6. And finally, calculate the gradients of the current layer, to pass them to the previous one. 
   
Every layer will receive the derivatives of the loss function for _each training example_. This way we can calculate the partial gradients of the cost function (the mean), for every weight and bias. 

### Step 1. Derivative of the loss function

We starth with the last layer. This one doesn't have any layer above it, so it will simply get the derivative of the loss function with respect of the output. That's easy:
$$\delta_i^l = \^{y} - y$$
This will give us a matrix with 32 rows (training examples) and one column (the gradient value). 

In [25]:
dl = out - sample_Y
dl.shape

(32, 1)

This is what we'll pass to the last layer. Every layer will update this value, calculating the gradient with respect of itself, and passing it to the nex one (what we defined as step 7).
### Step 2. Derivative of the activation function. 
If the layer has activation function, we have to multiply de loss by the derivative of ReLU. This is not a matrix multiplication, we'll perform a Hadamard product (noted with $\bigodot$ in math), wich is an element-wise multiplication of the matrix. To calculate the derivative of the ReLU function we need the results of the $z$ function. Luckily, we saved that numbers on the forward pass.
$$\delta^l = \delta^l \bigodot ReLU^\prime(z^l)$$
$$ReLU^\prime(z^l) = \begin{cases}
   0 &\text{if } z^l < 0 \\
   1 &\text{if } z^l > 0
\end{cases}$$
We see that performing this operations doesn't change the shape of the derivatives matrix. (We're accesing the last layer of the net with `net.layers[-1]`).

In [26]:
if net.layers[-1].activation == 'relu':
    dl *= np.greater(net.layers[-1].z, 0)
dl.shape

(32, 1)

### Step 3. Calculate the gradients for the weights
Recall that the last layer of our net has 4 weights (4 input), and only one neuron (one output). Later, when we update the weights, we will need to substract each gradient to the weight. That means that we need 4 values, corresponding to every weight. In fact, we will need a matrix with the same shape as the weights matrix (four rows and one column, as we only have one neuron in this layer)

In [30]:
net.layers[-1].weights.shape

(4, 1)

 And to calculate the gradient of the cost function for each weight of this layer, we need to multiply every gradient with its corresponding input for that layer, sum everything, and divide it by the batch size $m$:
$$\frac {\partial C}{\partial w^l} = \frac{1}{m}\sum_{j=1}^m input^l· \delta^l$$
Again, matrix multiplication can help us with this, as we need first to multiply and sum all the values of the training examples. But we have a problem, the shapes don't match!

In [27]:
print(f'Layer\'s inputs shape: {net.layers[-1].input.shape}, Gradients shape: {dl.shape}')


Layer's inputs shape: (32, 4), Gradients shape: (32, 1)


Fortunately, that's very easy to solve, we just have to transpose the input matrix...

In [29]:
net.layers[-1].input.T.shape

(4, 32)

...and now we have four rows, the same four rows of the weights matrix. The 32 values of each one of these rows will be multiplied by the 32 values of the column of the gradients matrix, summed together, and that value will be the sum of the derivatives of the examples _for each weight_.
$$\underbrace{\begin{bmatrix} i_{1.1} & i_{1.2} & ... & i_{1.32} \\
                  i_{2.1} & i_{2.2} & ... & i_{2.32} \\
                  i_{3.1} & i_{3.2} & ... & i_{3.32} \\
                  i_{4.1} & i_{4.2} & ... & i_{4.32} \\
 \end{bmatrix}}_{\text{Inputs.T}} \bullet
 \underbrace{\begin{bmatrix} d_1^l \\ d_2^l \\ ... \\ d_{32}^l \end{bmatrix}}_{\delta^l} = 
 \underbrace{\begin{bmatrix} d_{w1}^l \\ d_{w2}^l \\ d_{w3}^l \\ d_{w4}^l \end{bmatrix}}_{\delta_w^l}

In Python this looks much cleaner. Don't forget that we have to divide by the batch size:

In [31]:
dw_1 = np.dot(net.layers[-1].input.T, dl)/BATCH_SIZE
print('Weights gradients shape: ',  dw_1.shape, ', Weights shape: ', net.layers[-1].weights.shape)

Weights gradients shape:  (4, 1) , Weights shape:  (4, 1)


### Step 4. Calculate the gradients for the bias.
This step is much easier. Basically, it's just the mean of the gradients that we get form the next layer. We also check that it has the same shape as the bias matrix (in this case we just have 1 neuron, so 1 bias).

In [35]:
db_1 = dl.mean(axis=0)
print('Bias gradients shape: ',  db_1.shape, ', Bias shape: ', net.layers[-1].bias.shape)

Bias gradients shape:  (1,) , Bias shape:  (1, 1)


### Step 5. Update the parameters.
Here we multiply the gradients by the learning rate $\alpha$ that we decide, and substract that value from the weights and biases:
$$ w^l \gets w^l = w^l - \alpha·\delta_w^l$$
$$ b^l \gets b^l = b^l - \alpha·\delta_b^l$$


In [36]:
LR = 0.1
net.layers[-1].weights -= LR * dw_1
net.layers[-1].bias -= LR * db_1

### Step 6. Calculate the gradients of the current layer.

To calculate the gradient with respect of the layer, we have to make matrix multiplication between the loss and the transpose of the weights. This is the values that we pass to the next layer. In this case, the result will have 32 rows (training examples) and four columns. 

In [39]:
dx_1 = np.dot(dl, net.layers[-1].weights.T)
dx_1.shape

(32, 4)

Now we pass the gradient for this layer that we calculate it before (`dx`) to the next layer, and repeat the process. You can make it step by step as we did before, using `dx_1` as we used `dl` above, and just calling the parameters of `net.layers[-2]`. 

### Wrapping it up
We're going to encapsulate all this process in the methods `layer.backward()` and `net.backward()`:

In [42]:
@add_to_class(FullyConnected)
def backward(self, dl, batch_size):
    if self.activation == 'relu':
        dl *= np.greater(self.z, 0)
    dx = np.dot(dl, self.weights.T)
    self.dw = np.dot(self.input.T, dl) / batch_size
    self.db = dl.mean(axis=0)
    return dx

@add_to_class(Net)
def backward(self, dl, batch_size):
    for layer in reversed(self.layers):
        dl = layer.backward(dl, batch_size)

The code to actually update the weights and bias is contained in another class, `SGD` (for stochastic gradient descent). We do this because in other cases we might use other optimization methods, so it's good that we have it coded in another place rather than the actual layers. 

In [43]:
class Optimizer:
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def step(self):
        for layer in self.net.layers:
            self.update_params(layer)
    
    def update_params(layer):
        pass

class SGD(Optimizer):
    def update_params(self, layer):
        layer.weights -= self.lr * layer.dw
        layer.bias -= self.lr * layer.db

## Training loop
Let's summarize everything. What we did:
1. Take a __sample__ from the training dataset.
2. Calculate the expected output with a __forward pass__ through the net.
3. Calculate the derivative of the __loss function__.
4. Perform a __backward pass__ to calculate the gradients for each parameter.
5. __Update__ the parameters. 

All this process is called an _epoch_. In deep learning, we repeat this many times, until we have good results. 
Let wrap everything in a for loop, and see how the net performs with the fake dataset. 

In [54]:
EPOCHS = 50
losses = []
net = Net((2, 3, 4, 1))
optim = SGD(net, LR)
for epoch in range(EPOCHS):
    indices = np.random.randint(len(X), size=(BATCH_SIZE))
    sample_X = X[indices]
    sample_Y = Y[indices]
    out = net(sample_X)
    sample_loss = MSE(sample_Y, out)
    losses.append(sample_loss)
    d_loss = (out - sample_Y)
    net.backward(d_loss, BATCH_SIZE)
    optim.step()

Now we plot the change in the loss function for the samples in each epoch

In [55]:
import plotly.express as px
px.line(losses)

In [60]:
print(f'True value: {sample_Y[0]}, Predicted value{out[0]}')

True value: [106.74864048], Predicted value[93.62023766]


We see that the loss starts with high values and quickly stabilizes at around 150. The predicted value is not that far from the true value for the first example, so we can conclude that the model, even if not perfect, it works.

# Classifying MNIST dataset

We'll see if our model can handle the MNIST dataset and correclty classify images of handwritten numbers. We're gonna make a new network, with more layers and neurons. We're also gonna split the data in train and test sets, common practice in deep learning models.

In [61]:
import plotly.express as px
import numpy as np
from src.model import Net
from src.optim import SGD
from torchvision.datasets import MNIST
from torch.utils.data import random_split

data = MNIST('./data', train=False)
train, test = random_split(data, [0.8,0.2])

In [62]:
def process_data(dataset):
    X = np.zeros((len(dataset), 28*28))
    Y = np.zeros((len(dataset), 10))
    for n in range(len(dataset)):
        X[n] = np.asarray(dataset[n][0]).flatten()/255
        Y[n][dataset[n][1]] = 1
    return X, Y

X_train, Y_train = process_data(train)
X_test, Y_test = process_data(test)


For classifying problems, is vey useful to use the _accuracy_ to check if the model is working well. The accuracy is a metric that tells us how often a network classifies correctly a data point. Let's define a method to calculate it

In [64]:
def accuracy(exp, pred):
    exp_number = np.argmax(exp, axis=1)
    pred_number = np.argmax(pred, axis=1)
    true_pred = (exp_number == pred_number).sum()
    return true_pred/len(exp)

In [65]:
LR_MNIST = 0.01
BATCH_SIZE_MNIST = 32
EPOCHS_MNIST = 10000

costs = {'train': [], 'test': []}
accuracies = {'train': [], 'test': []}
net = Net((28*28, 15, 10), last_relu=False)
optim = SGD(net, LR_MNIST)

for epoch in range(EPOCHS_MNIST):
    # Sample minibatch:
    indices = np.random.randint(len(X_train), size=(BATCH_SIZE_MNIST))
    sample_X = X_train[indices]
    sample_Y = Y_train[indices]
    train_out = net(sample_X)
    # Update net
    d_loss = (train_out - sample_Y)
    net.backward(d_loss, BATCH_SIZE_MNIST)
    optim.step()
    # Save loss
    test_out = net(X_test)
    sample_cost = MSE(sample_Y, train_out)
    test_cost = MSE(Y_test, test_out)
    costs['train'].append(sample_cost)
    costs['test'].append(test_cost)
    # Save accuracy
    accuracies['train'].append(accuracy(sample_Y, train_out))
    accuracies['test'].append(accuracy(Y_test, test_out))

In [73]:
px.line(costs, 
        title='Cost function',
        labels={'index':'epoch',
                'value':'cost',
                'variable':'set'}
        )

In [74]:
px.line(accuracies, 
        title='Accuracy',
        labels={'index':'epoch',
                'value':'accuracy',
                'variable':'set'}
        )

Let's see it with one of the examples:

In [75]:
px.imshow(X_test[432].reshape(28,28), color_continuous_scale='greys')

In [79]:
print(f'True value: {np.argmax(Y_test[432])}, Predicted value{np.argmax(test_out[432])}')


True value: 9, Predicted value9
